In [7]:
import os
import torch
import numpy as np
import os
#import cv2
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
np.set_printoptions(suppress=True)

In [8]:
radar_root = "/workspaces/Radar-Camera-Fusion-Detection/WaterScenes/data/radar"
save_radar_map_root = "/workspaces/Radar-Camera-Fusion-Detection/WaterScenes/data/radar/REVP_map"
if not os.path.exists(save_radar_map_root):
    os.makedirs(save_radar_map_root)

In [9]:
radar_files = os.listdir(radar_root)
len(radar_files)

51

In [10]:
# u and v represents the x and y on 2D image plane
features_list = ['range', 'elevation', 'doppler', 'power', 'u', 'v']

In [11]:
resolution = 160
original_width = 1920
original_height = 1080

x_scale = original_width / resolution
y_scale = original_height / resolution

In [12]:
for i in tqdm(range(len(radar_files))):
    example_file = os.path.join(radar_root, radar_files[i])
    
    # 파일이 존재하는지 확인 (추가적인 예외처리)
    if not os.path.isfile(example_file):
        print(f"Warning: File {example_file} not found, skipping...")
        continue

    example_radar_points = pd.read_csv(example_file)[features_list].to_numpy()
    example_radar_map = np.zeros((len(features_list)-2, resolution, resolution))

    for channel in range(len(features_list)-2):
        for line in example_radar_points:
            try:
                row_index = int(line[-2] / x_scale)
                column_index = int(line[-1] / y_scale)
                if example_radar_map[channel][row_index][column_index] != 0 and row_index >= 1:
                    row_index -= 1
                
                example_radar_map[channel][row_index][column_index] = line[channel]
            except Exception as e:
                print(f"Error processing point {line}: {e}")
                continue

    example_radar_map = example_radar_map.transpose(0, 2, 1)

    # 저장할 파일 경로
    save_file_path = os.path.join(save_radar_map_root, radar_files[i][:-4] + '.npz')
    
    # 파일 저장
    np.savez_compressed(save_file_path, example_radar_map)

print("Processing completed successfully!")


100%|██████████| 51/51 [00:00<00:00, 100.29it/s]

Processing completed successfully!


In [13]:
data = np.load("/workspaces/Radar-Camera-Fusion-Detection/WaterScenes/data/radar/REVP_map/000001.npz")
keys = data.files
print(data['arr_0'].shape)

(4, 160, 160)
